In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['cell_images']


In [2]:
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from PIL import Image
import cv2

Using TensorFlow backend.


In [3]:
img=[]
labels=[]

In [4]:
Parasitized=os.listdir("../input/cell_images/cell_images/Parasitized/")
for a in Parasitized:
    try:
        image=cv2.imread("../input/cell_images/cell_images/Parasitized/"+a)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        img.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")



In [5]:
Uninfected=os.listdir("../input/cell_images/cell_images/Uninfected/")
for b in Uninfected:
    try:
        image=cv2.imread("../input/cell_images/cell_images/Uninfected/"+b)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        img.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")

In [6]:
len(img)

27558

In [7]:
len(labels)

27558

In [8]:
img=np.array(img)
labels=np.array(labels)

In [9]:
img.shape

(27558, 50, 50, 3)

In [10]:
len_data=len(img)

In [11]:
s=np.arange(img.shape[0])
np.random.shuffle(s)
img=img[s]
labels=labels[s]

In [12]:
(x_train,x_test)=img[(int)(0.1*len_data):],img[:(int)(0.1*len_data)]
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255
(y_train,y_test)=labels[(int)(0.1*len_data):],labels[:(int)(0.1*len_data)]

In [13]:
y_train=np_utils.to_categorical(y_train)

In [14]:
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [15]:
y_test=np_utils.to_categorical(y_test)

In [16]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(250,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 64)          0         
__________

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(x_train,y_train,batch_size=50,epochs=30,verbose=1,validation_split=0.1)

Train on 22322 samples, validate on 2481 samples
Epoch 1/30
22322/22322 [==============================] - 7s 314us/step - loss: 0.3607 - acc: 0.8318 - val_loss: 0.1756 - val_acc: 0.9440
Epoch 2/30
22322/22322 [==============================] - 3s 119us/step - loss: 0.1649 - acc: 0.9475 - val_loss: 0.1578 - val_acc: 0.9480
Epoch 3/30
22322/22322 [==============================] - 3s 120us/step - loss: 0.1460 - acc: 0.9526 - val_loss: 0.1440 - val_acc: 0.9528
Epoch 4/30
22322/22322 [==============================] - 3s 119us/step - loss: 0.1340 - acc: 0.9549 - val_loss: 0.1499 - val_acc: 0.9549
Epoch 5/30
22322/22322 [==============================] - 3s 123us/step - loss: 0.1268 - acc: 0.9582 - val_loss: 0.1325 - val_acc: 0.9565
Epoch 6/30
22322/22322 [==============================] - 3s 120us/step - loss: 0.1167 - acc: 0.9603 - val_loss: 0.1297 - val_acc: 0.9585
Epoch 7/30
22322/22322 [==============================] - 3s 121us/step - loss: 0.1090 - acc: 0.9619 - val_loss: 0.1185 - v

In [19]:
pred=model.predict(x_test)

In [20]:
pred

array([[7.6832030e-06, 9.9999237e-01],
       [2.9860377e-07, 9.9999964e-01],
       [1.0000000e+00, 9.7492780e-26],
       ...,
       [3.4082543e-06, 9.9999654e-01],
       [2.7527697e-06, 9.9999726e-01],
       [1.0000000e+00, 8.8041593e-30]], dtype=float32)

In [21]:
pred=np.argmax(pred,axis=1)

In [22]:
pred=np_utils.to_categorical(pred)

In [23]:
pred

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [24]:
y_test

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.9549909255898367